# Chapter 8a — ReAct Agent (Reason + Act)

**Pattern:** A single agent that interleaves reasoning and tool use — it thinks about what to do, acts, observes the result, and re-plans after every step.

```
Goal → [Think → Act → Observe → Think → Act → Observe → ...] → Final Answer
```

### How This Builds on Earlier Chapters

| Chapter | Pattern | What we learned | How ReAct extends it |
|---------|---------|-----------------|---------------------|
| Ch 5 | Tool Use | Agents call tools to get live data | ReAct chains **multiple tool calls** with reasoning between each |
| Ch 7 | Sequential | Agents run in a fixed order | ReAct has **no fixed order** — the agent decides what's next after each observation |
| Ch 6 | Reflection | A reviewer checks output against rules | ReAct reflects **inline** — every Thought step is a mini-reflection |

### When to Use ReAct vs Other Patterns

- **Exploratory tasks** where you don't know how many steps you'll need
- **Search-heavy tasks** where each result changes what to do next
- **Short-horizon tasks** (2–5 steps) where a full plan is overkill
- When **all reasoning should be visible** in one agent's trace

### The Scenario

You're a **tech lead** evaluating whether to adopt Rust for a performance-critical microservice. You need to research Rust's memory model, compare it to your team's current Go stack, write up a recommendation, and verify the key claims. The research is *exploratory* — what you learn from one search changes what you search for next.

In [7]:
import os
import glob
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()
assert os.environ.get("GOOGLE_API_KEY"), "Set GOOGLE_API_KEY first"
print("Google API Key set:", bool(os.environ.get("GOOGLE_API_KEY")))

Google API Key set: True


In [8]:
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import FunctionTool
from google.genai import types

---
## Tools — Mock Knowledge Base + Notes System

We use **deterministic mocks** (same lesson from Ch 6: mock internals for reproducibility). The mock search returns realistic results that create an interesting research path — each result raises new questions.

| Tool | Purpose | Why the agent needs it |
|------|---------|------------------------|
| `web_search` | Look up technical topics | Each search result informs what to search next |
| `write_note` | Save a research finding | Builds up a knowledge base across steps |
| `list_notes` | See what's been researched | Helps the agent avoid redundant searches |

In [9]:
def web_search(query: str) -> str:
    """Search the web for technical information. Returns a summary of findings."""
    # Deterministic mock — realistic results that create an interesting research path
    knowledge_base = {
        "rust ownership": (
            "Rust's ownership system guarantees memory safety without a garbage collector. "
            "Every value has exactly one owner. When the owner goes out of scope, the value is dropped. "
            "Borrowing rules: you can have either one mutable reference OR multiple immutable references. "
            "This eliminates data races at compile time. Zero-cost abstractions mean no runtime overhead."
        ),
        "rust vs go performance": (
            "Benchmarks show Rust is 2-5x faster than Go for CPU-bound tasks due to zero-cost abstractions "
            "and no GC pauses. Go excels in developer productivity and has faster compile times. "
            "Rust's memory model eliminates GC pause spikes — critical for p99 latency requirements. "
            "Go's goroutines make concurrent I/O simpler but Rust's async/await with tokio matches throughput."
        ),
        "rust learning curve": (
            "Rust has a steep learning curve — most teams report 3-6 months to productivity. "
            "The borrow checker causes friction for developers from GC languages. "
            "However, Rust's compiler errors are highly informative and guide fixes. "
            "Teams report fewer production bugs after the learning period — the compiler catches issues upfront."
        ),
        "rust adoption enterprise": (
            "Major adopters: AWS (Firecracker), Microsoft (Windows kernel), Cloudflare (edge workers), "
            "Discord (switched from Go to Rust for latency-sensitive services). "
            "Discord's case study: Go's GC pauses caused latency spikes every 2 minutes. "
            "After rewriting in Rust, p99 latency dropped from 300ms to 10ms. Memory usage halved."
        ),
        "go microservice": (
            "Go strengths for microservices: fast compilation, simple concurrency model (goroutines), "
            "small binary sizes, excellent standard library for HTTP/networking. "
            "Weakness: GC pauses (typically 1-3ms but can spike to 10ms+ under heap pressure). "
            "Go 1.22 improved GC but latency-sensitive services still need careful tuning."
        ),
    }
    for key, result in knowledge_base.items():
        if key in query.lower():
            return result
    return f"Search results for '{query}': Several relevant articles found covering this topic."


def write_note(title: str, content: str) -> str:
    """Save a research note to the local filesystem."""
    path = f"/tmp/react_note_{title.replace(' ', '_').lower()}.txt"
    with open(path, "w") as f:
        f.write(f"# {title}\n\n{content}")
    return f"Note saved: {path} ({len(content)} chars)"


def list_notes() -> str:
    """List all saved research notes."""
    files = glob.glob("/tmp/react_note_*.txt")
    if not files:
        return "No notes saved yet."
    result = []
    for f in sorted(files):
        with open(f) as fh:
            first_line = fh.readline().strip()
        result.append(f"  {f} — {first_line}")
    return "Saved notes:\n" + "\n".join(result)


print("Tools defined: web_search, write_note, list_notes")

Tools defined: web_search, write_note, list_notes


---
## The ReAct Agent

A **single `LlmAgent`** with an instruction that encourages step-by-step reasoning **between** tool calls.

The key insight: we do **not** ask the model to write out "Action:" or "Observation:" labels — that causes Gemini to role-play tool use in plain text instead of actually calling tools through ADK. Instead, we tell it to:

1. **Think before each tool call** — explain what you're doing and why
2. **Use the actual tools** — ADK handles the function calling protocol
3. **Reflect after each result** — update your approach based on what you learned
4. **Call tools multiple times** — don't try to answer everything in one shot

The `FunctionTool` wrapper (from `google.adk.tools`) converts plain Python functions into tools that ADK can expose to Gemini. ADK reads the function signature + docstring to generate the JSON schema.

In [10]:
REACT_INSTRUCTION = """
You are a research agent that reasons step-by-step and uses tools to gather information.

Your process for EVERY task:
1. Think about what information you need and explain your reasoning briefly.
2. Call the appropriate tool (web_search, write_note, or list_notes).
3. After getting the result, reflect on what you learned and decide what to do next.
4. Repeat steps 1-3 as many times as needed — do NOT try to answer in one step.
5. When you have enough information, save your key findings using write_note.
6. Finally, provide your complete answer.

IMPORTANT RULES:
- You MUST actually call the tools — do not simulate or role-play tool usage in text.
- Call web_search multiple times for different aspects of the research.
- After each search result, think about what ELSE you need to learn before proceeding.
- Save important findings as notes before giving your final answer.
- Be thorough — research at least 3 different aspects before concluding.
"""

react_agent = LlmAgent(
    name="ReActResearcher",
    model="gemini-2.0-flash",
    instruction=REACT_INSTRUCTION,
    description="Single agent that reasons and acts in an interleaved loop using real tool calls.",
    tools=[
        FunctionTool(web_search),
        FunctionTool(write_note),
        FunctionTool(list_notes),
    ],
)

print("ReAct agent ready with tools:", [t.name for t in react_agent.tools])

ReAct agent ready with tools: ['web_search', 'write_note', 'list_notes']


---
## Runner — Streaming the ReAct Loop

We stream events so you can **watch the reasoning happen** — every Thought, every tool call, every observation is visible. This is the key advantage of ReAct: the entire reasoning trace is in one agent, making it easy to debug.

Compare this to Ch 7's multi-agent patterns where reasoning is split across agents. ReAct trades modularity for transparency.

In [11]:
async def run_react(goal: str, user_id: str = "user_001"):
    """Run the ReAct agent on a goal and stream the reasoning trace."""
    print(f"{'='*60}")
    print(f"GOAL: {goal}")
    print(f"{'='*60}")

    session_service = InMemorySessionService()
    runner = Runner(
        agent=react_agent,
        app_name="react_research",
        session_service=session_service,
    )

    session = await session_service.create_session(
        app_name="react_research", user_id=user_id,
    )

    content = types.Content(
        role="user",
        parts=[types.Part.from_text(text=goal)],
    )

    step_count = 0
    async for event in runner.run_async(
        user_id=user_id, session_id=session.id, new_message=content,
    ):
        if event.content:
            for part in event.content.parts:
                # Tool calls — the "Act" in ReAct
                if hasattr(part, "function_call") and part.function_call:
                    fc = part.function_call
                    step_count += 1
                    print(f"\n  [{step_count}] TOOL CALL: {fc.name}({dict(fc.args)})")

                # Tool results — the "Observe" in ReAct
                elif hasattr(part, "function_response") and part.function_response:
                    fr = part.function_response
                    result_str = str(fr.response)[:200]
                    print(f"      RESULT: {result_str}{'...' if len(str(fr.response))>200 else ''}")

                # Text — the "Think" in ReAct
                elif part.text and part.text.strip():
                    print(f"\n{part.text}")

        if event.is_final_response():
            print(f"\n{'='*60}")
            print(f"DONE — {step_count} tool calls")
            print(f"{'='*60}")

print("Runner function defined: run_react()")

Runner function defined: run_react()


---
## Run It — Exploratory Research Task

Watch the agent's reasoning unfold. Notice how each search result **changes what it searches for next** — this is the core strength of ReAct over fixed sequential pipelines.

In [12]:
await run_react(
    "Research Rust's ownership model and compare it to Go for building "
    "latency-sensitive microservices. Look into real-world adoption cases. "
    "Save your key findings as notes, then give me a recommendation "
    "on whether our team should adopt Rust for our new payment gateway service."
)

GOAL: Research Rust's ownership model and compare it to Go for building latency-sensitive microservices. Look into real-world adoption cases. Save your key findings as notes, then give me a recommendation on whether our team should adopt Rust for our new payment gateway service.



Okay, I need to research Rust's ownership model, compare it to Go for building latency-sensitive microservices, and look into real-world adoption cases. Then, I will provide a recommendation on adopting Rust for a new payment gateway service.

First, I'll start by researching Rust's ownership model.


  [1] TOOL CALL: web_search({'query': 'Rust ownership model'})
      RESULT: {'result': "Rust's ownership system guarantees memory safety without a garbage collector. Every value has exactly one owner. When the owner goes out of scope, the value is dropped. Borrowing rules: yo...

The ownership model seems to be a core feature ensuring memory safety and preventing data races at compile time. Now, I'll research Go's memory management and concurrency model and compare that to Rust.


  [2] TOOL CALL: web_search({'query': 'Go memory management and concurrency model vs Rust'})
      RESULT: {'result': "Search results for 'Go memory management and concurrency model vs Rust': Several relevant 

---
## Deep Dive: What Actually Happened Inside the Loop

To understand the ReAct pattern, you need to see **three actors** working together: your Python code (the tools), ADK (the middleman), and Gemini (the brain). Your `run_react()` function is just a window — it prints events as they stream by but doesn't drive the reasoning.

### The Three Actors

```
Your Code                          ADK (invisible)                    Gemini (the brain)
─────────                          ───────────────                    ──────────────────
                                   sends message + tools ──────────►  
                                                                      thinks: "I need to search"
                                   ◄────────── function_call: web_search("Rust ownership model")
                                   runs YOUR Python function
                                   sends result back ──────────────►
                                                                      thinks: "good, now I need Go info"
                                   ◄────────── function_call: web_search("Go memory management...")
                                   runs YOUR Python function
                                   sends result back ──────────────►
                                                                      thinks: "that was vague, retry"
                                   ◄────────── function_call: web_search("Go garbage collection...")
                                   ...repeats until satisfied...
                                   ◄────────── text only (final answer)
you print each event
as it streams by
```

**ADK calls Gemini multiple times** inside a single `runner.run_async()` call. Each time Gemini returns a `function_call`, ADK runs your Python function and sends the result back. This continues until Gemini responds with text only (no more tool calls).

### Tracing Each Step Against the Knowledge Base

Our mock `web_search` has exactly **5 keys** that return real data. Everything else hits a generic fallback:

```python
knowledge_base = {
    "rust ownership"           →  detailed Rust ownership explanation
    "rust vs go performance"   →  benchmarks, p99 latency comparison
    "rust learning curve"      →  3-6 months, borrow checker friction
    "rust adoption enterprise" →  AWS, Discord, Cloudflare case studies
    "go microservice"          →  Go strengths, GC pause weakness
}
# Anything else returns: "Search results for '...': Several relevant articles found."
```

The matching logic is `if key in query.lower()` — the key must appear as a substring in the query.

---

**Step [1]** — Query: `"Rust ownership model"`

Does `"rust ownership"` appear in this query? **Yes.** Returns detailed Rust ownership explanation. Gemini gets rich data and proceeds.

**Step [2]** — Query: `"Go memory management and concurrency model vs Rust"`

Check each key: `"rust ownership"` in query? No. `"rust vs go performance"`? No. `"go microservice"`? No. **None match** → generic fallback.

Gemini compares this vague result to Step 1's rich response and reasons: *"That's not very specific. Let me refine."*

**Step [3]** — Query: `"Go garbage collection vs Rust ownership"`

`"rust ownership"` in `"go garbage collection vs rust ownership"`? **Yes!** But this returns the *Rust* explanation again — not the Go info Gemini wanted. Gemini says: *"Still pretty generic."*

**Step [4]** — Query: `"Go garbage collection latency vs Rust"`

No key matches → **generic fallback**. Gemini: *"Still not great."*

**Step [5]** — Query: `"Rust vs Go latency-sensitive microservices"`

Close! But the key is `"rust vs go performance"` and the query says `"rust vs go latency-sensitive microservices"`. No exact substring match → **generic fallback**.

**Step [6]** — Query: `"performance comparison Rust vs Go microservices"`

`"go microservice"` in `"performance comparison rust vs go microservices"`? **Yes!** (The query contains `"go microservices"` which includes the substring `"go microservice"`.) Finally returns real Go data: GC pauses, goroutines, compilation speed.

**Steps [7-8]** — More searches, generic fallbacks. Gemini decides it has enough from steps [1], [3], and [6].

**Steps [9-10]** — Searches for payment gateway adoption cases. No keys match → fallback. Gemini acknowledges the gap and moves on.

**Steps [11-12]** — Gemini saves its findings as notes using `write_note`, then delivers the final recommendation.

---

### The Key Insight

**Your code never told Gemini to retry or refine its queries.** The `run_react()` function just prints events. The retry behavior is Gemini's own reasoning — it saw a vague result, judged it insufficient compared to earlier rich results, and autonomously decided to try a different query. That's the core of the ReAct pattern: the LLM self-corrects after every observation.

This is also why ReAct made **12 tool calls** instead of the 5 you might expect (one per knowledge base entry). The mock's limited key matching forced Gemini to be creative with its queries — and you got to watch it adapt in real time.

---
## Try a Different Goal

The same agent works on any exploratory task. Try changing the goal to see how the reasoning path changes.

In [ ]:
# Uncomment and run with your own goal:
# await run_react("Research the pros and cons of on-device AI inference vs cloud inference. Save a comparison note.")

---
## Key Takeaways

### ReAct vs Patterns from Earlier Chapters

| Dimension | Ch 5 Tool Use | Ch 7 Sequential | Ch 6 Reflection | **Ch 8a ReAct** |
|-----------|---------------|-----------------|-----------------|----------------|
| Tool calls | One | Fixed chain | Separate reviewer | **Many, dynamic** |
| Planning | None | Pre-defined | Post-hoc review | **Inline, after every step** |
| Execution order | Single step | Fixed A→B→C | Generate→Review loop | **Emergent from reasoning** |
| Debugging | Easy | Trace across agents | Two-agent trace | **Single agent trace** |
| Best for | Simple lookups | Assembly-line tasks | Compliance checking | **Exploratory research** |

### Critical Design Lesson: Don't Ask the Model to Role-Play Tool Use

The original ReAct paper uses a `Thought: / Action: / Observation:` text format. But with ADK, this is a **trap** — if you put those labels in the system prompt, Gemini will *write out* the tool calls as text instead of *actually calling* the tools via ADK's function calling protocol.

The fix: tell the agent to **use the tools directly** and reason between calls. ADK handles the function calling mechanics. You get the same Think→Act→Observe loop, but through real tool invocations that show up as `function_call` / `function_response` events.

### Design Decisions

- **Simple, direct instruction** — No "Thought/Action/Observation" labels. Just tell the agent to reason, use tools, and reflect after each result. ADK's function calling does the rest.
- **`FunctionTool` wrapper** — Converts plain Python functions into ADK tools. ADK reads the function signature + docstring to generate the JSON schema Gemini needs.
- **Deterministic mocks** — Same principle as Ch 6: mock data makes the research path reproducible and testable.
- **No plan commitment** — Unlike Ch 8b (Plan-and-Execute), ReAct doesn't commit to a plan upfront. Each step is decided *after* seeing the previous result.

### When NOT to Use ReAct

- **Structured multi-step tasks** (6+ steps) — Use Plan-and-Execute (Ch 8b) instead
- **Tasks needing an auditable plan** — ReAct's plan is implicit in the trace, not a standalone artifact
- **Parallelizable work** — ReAct is inherently serial. Use ParallelAgent (Ch 3) for concurrent execution